In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import pickle
from konlpy.tag import *

### 데이터 전처리

In [8]:

# 데이터를 읽어온다.
df = pd.read_csv('naver_star_data.csv')
# df.head()
# 데이터 전처리
# 후기데이터 전처리
def text_preprocessing(text) :
    # 관람객으로 시작하면 앞에 3글자를 없앤다.
    if text.startswith('관람객') :
        new_str = text[3:]
        return new_str
    else :
        return text
    
df['text'] = df['text'].apply(text_preprocessing)

# 평점 전처리
def score_preprocessing(text) :
    value = int(text)
    if value <= 5 :
        return '0'
    else :
        return '1'
    
df['star'] = df['star'].apply(score_preprocessing)

df.to_csv('naver_star_data_pre.csv', index=False, encoding='utf-8-sig')

print('전처리 완료')

전처리 완료


### 학습데이터와 검증 데이터로 나눈다

In [9]:
# 데이터를 읽어온다.
df = pd.read_csv('naver_star_data_pre.csv')
# 7:3 비율로 나눈다.
text_list = df['text'].tolist()
star_list = df['star'].tolist()

text_train, text_test, star_train, star_test = train_test_split(text_list, star_list, test_size=0.3)
# 저장한다.
dic_train = {
    'text' : text_train,
    'star' : star_train
}
dic_test = {
    'text' : text_test,
    'star' : star_test
}

df_train = pd.DataFrame(dic_train)
df_test = pd.DataFrame(dic_test)

df_train.to_csv('train.csv', index=False, encoding='utf-8-sig')
df_test.to_csv('test.csv', index=False, encoding='utf-8-sig')

print('저장완료')


저장완료


### 학습

In [2]:
# 데이터를 읽어온다.
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# 입력데이터와 결과 데이터로 나눈다.
X_train = train_df['text'].tolist()
y_train = train_df['star'].tolist()

X_test = test_df['text'].tolist()
y_test = test_df['star'].tolist()

# 모델 객체 생성
# 문자열 데이터를 단어사전을 만들어 백터화 시킨다.
okt = Okt()
def text_tokenizer(text) :
    return okt.morphs(text)

model1 = TfidfVectorizer(lowercase=False, tokenizer=text_tokenizer)
# 분류 모델
model2 = LogisticRegression(C=10.0, penalty='l2')

pipe = Pipeline([('vect', model1), ('clf', model2)])

# 학습한다.
import time
start_time = time.time()
print('학습시작')
pipe.fit(X_train, y_train)
print('학습종료')
end_time = time.time()
print(f'총 학습 시간 : {end_time - start_time}')

# 학습 정확도를 측정한다.
# 학습된 모델로 예측 결과를 계산한다.
y_pred = pipe.predict(X_test)
# 예측 결과와 진짜 결과를 비교하여 정확도를 계산한다.
print(f'정확도 : {accuracy_score(y_test, y_pred)}')

# 저장한다.
with open('pipe.dat', 'wb') as fp :
    pickle.dump(pipe, fp)
    
print('저장완료')

학습시작
학습종료
총 학습 시간 : 245.6600000858307
정확도 : 0.9173451127698334
저장완료


### 활용

In [ ]:
# 파일로 저장된 모델을 복원한다.
with open('pipe.dat', 'rb') as fp :
    pipe = pickle.load(fp)
    
text = input('리뷰를 작성해주세요')

str1 = [text]
# 예측 정확도를 계산
r1 = np.max(pipe.predict_proba(str1) * 100)
# 예측 결과
r2 = pipe.predict(str1)[0]

if r2 == 1 :
    print('긍정적인 리뷰')
else :
    print('부정적인 리뷰')
    
print(f'정확도 : {r1}')
